# 2. Leptons Selection

First, we import the libraries for reading and manipulating our ROOT files. We parametrze the analysis with path to the data, the name of the column (obtained from the CERN page of the datasets)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# List of the ROOT files downloaded.
DATA_FILES = {
    "DoubleMuon_B": "../../data/12365/Run2012B_DoubleMuParked.root",
    "DoubleMuon_C": "../../data/12366/Run2012C_DoubleMuParked.root",
    "DoubleElectron_B": "../../data/12367/Run2012B_DoubleElectron.root",
    "DoubleElectron_C": "../../data/12368/Run2012C_DoubleElectron.root"
}

# Number of events to load
MAX_EVENTS = 300000

try:
    import uproot # For reading CERN ROOT files
    import vector # For fast, correct Lorentz Vector calculations
    import awkward as ak # For manipulating variable-length lists
    print("Success: uproot, vector, and awkward are loaded.")

    # Necessary branches (columns) for each lepton type.
    MUON_BRANCHES = ["event", "Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_charge", "Muon_pfRelIso03_all"]
    ELECTRON_BRANCHES = ["event", "Electron_pt", "Electron_eta", "Electron_phi", "Electron_mass", "Electron_charge", "Electron_pfRelIso03_all"]

except ImportError:
    print("Error: uproot, vector, or awkward not found. Please install these libraries to continue.")
    exit()

The next function  primary goal is to transform our data into a **flat table** where each row represents a single lepton.

### 1. Data Type Determination (The Dispatcher)

The function first acts as a **dispatcher**, determining the required physics schema based on the input file's identifier (`file_key`):

It checks for `"DoubleMuon"` or `"DoubleElectron"` in the `file_key`. This is a way to get the **lepton flavor** ($\mu$ or $e$) and select the correct set of **TTree branches** (e.g., `Muon_pt` vs. `Electron_pt`).

Then we set the `lepton_prefix` (e.g., `"Muon"`) and the **PDG ID** (`flavor_pdg`, 13 or 11), ensuring the output DataFrame is properly annotated.

### 2. Event Array Retrieval (The Reader)

Using the `uproot` library, the function streams data from the ROOT file's `Events` TTree: It only reads the necessary branches (defined by `MUON_BRANCHES` or `ELECTRON_BRANCHES`), minimizing I/O and memory overhead.

The data is loaded into an **Awkward Array (`ak`)**. This choice is crucial because it inherently handles the **variable-length lists** of objects (leptons) within each event—the "jagged" structure.

### 3. Flattening and Event ID Reconstruction (The Core Transformation)

This is the central logic, converting the Awkward Array into a flat, tabular format ready for Pandas.

It calculates `lepton_counts` by getting the length of the list of $p_T$ values (`ak.num(raw_data[f'{lepton_prefix}_pt'])`) for each event. This count is essential for the row-wise mapping.

Then, the scalar **`event` ID** is converted to a NumPy array and  `np.repeat(event_ids_np, lepton_counts_np)` is used to **duplicate the event ID** for every lepton it contains. If event 123 has 3 leptons, the ID 123 appears 3 times. This creates the primary key for the flat table.

For every required branch (`pt`, `eta`, `iso`, etc.):
    1.  `ak.flatten(...)` converts the jagged array (list of lists) into a single, contiguous 1D array of particle properties.
    2.  `ak.to_numpy(...)` extracts the raw values, preparing them for the Pandas DataFrame construction.

### 4. Final Data Frame Assembly

The collected 1D arrays are assembled into the final Pandas DataFrame (`df`).

It explicitly iterates over kinematic columns (`pt`, `eta`, etc.) to enforce a uniform `np.float64` data type. This is good practice to prevent downstream issues with vector arithmetic or type mismatches when concatenating multiple files.

The pre-determined `flavor_pdg` (13 or 11) is appended as a new column, to provide a permanent **particle identification** tag.

In [ ]:

def load_data_from_file(file_path, file_key, max_events):
    """
    Loads specific lepton data (Muon or Electron) based on the trigger file type (file_key).
    Returns a flattened DataFrame for that file.
    """

    # 1. Determine the branches to load based on the trigger type
    if ("DoubleMuon" in file_key) or ("4mu" in file_key):
        lepton_prefix = "Muon"  # Capitalized to match branch names
        branches = MUON_BRANCHES
        flavor_pdg = 13 # Muon
    elif ("DoubleElectron" in file_key) or ("4e" in file_key):
        lepton_prefix = "Electron" # Capitalized to match branch names
        branches = ELECTRON_BRANCHES
        flavor_pdg = 11 # Electron
    else:
        print(f"Warning: Unrecognized file type ({file_key}). Skipped.")
        return pd.DataFrame()

    try:
        # Open the ROOT file and read the 'Events' tree (TTree)
        with uproot.open(file_path) as file:
            tree = file["Events"]

            # Read only the necessary branches for this file
            raw_data = tree.arrays(
                branches,
                entry_stop=max_events,
                library="ak"
            )
            print(f"Step 1: Raw Data read OK")

            # --- CONVERSION TO 'FLATTENED' FORMAT (CORRECTED) ---

            # Initialize the dictionary for the flat DataFrame
            flattened_data = {}

            # 1. Handle Event ID (Repeat the Event ID for each lepton it contains)
            event_ids = raw_data['event']

            # We use the pT branch to determine the number of leptons per event
            lepton_counts = ak.num(raw_data[f'{lepton_prefix}_pt'])

            # Robust conversion to NumPy for np.repeat
            lepton_counts_np = ak.to_numpy(lepton_counts).astype(np.int64)
            event_ids_np = ak.to_numpy(event_ids).astype(np.int64)

            # Repeat the Event ID as many times as there are leptons
            flattened_data['event_id'] = np.repeat(event_ids_np, lepton_counts_np)

            # Diagnostic
            print(f"Diagnostic: Read {len(raw_data)} events. Number of {lepton_prefix}s in the first event: {lepton_counts_np[0] if len(lepton_counts_np) > 0 else 0}")

            # 2. Flatten the lepton data (pt, eta, phi, etc.)
            # We don't loop because renaming is complex with capitalization.

            # Kinematics
            flattened_data['pt'] = ak.to_numpy(ak.flatten(raw_data[f'{lepton_prefix}_pt']))
            flattened_data['eta'] = ak.to_numpy(ak.flatten(raw_data[f'{lepton_prefix}_eta']))
            flattened_data['phi'] = ak.to_numpy(ak.flatten(raw_data[f'{lepton_prefix}_phi']))
            flattened_data['mass'] = ak.to_numpy(ak.flatten(raw_data[f'{lepton_prefix}_mass']))
            flattened_data['charge'] = ak.to_numpy(ak.flatten(raw_data[f'{lepton_prefix}_charge']))

            # Isolation
            iso_key = f'{lepton_prefix}_pfRelIso03_all'
            flattened_data['iso'] = ak.to_numpy(ak.flatten(raw_data[iso_key]))

            df = pd.DataFrame(flattened_data)
            print(f"Step 2: Flatten OK")

            # --- KEY CORRECTION: ENFORCE NUMERIC TYPING FOR 'VECTOR' ---
            kinematic_cols = ['pt', 'eta', 'phi', 'mass']
            for col in kinematic_cols:
                # Ensure kinematic columns are of standard float64 type
                if col in df.columns:
                    df[col] = df[col].astype(np.float64)

            # Add the flavor identification column (PDG ID)
            df['flavor'] = flavor_pdg
            print(f"Step 3: ID OK")

            for col in kinematic_cols:
                print(f"DEBUG: Type/Dtype of df['{col}'].values: {type(df[col].values)} / {df[col].values.dtype}")

            print(f"Success: {len(df)} leptons loaded from {file_key}.")
            return df

    except Exception as e:
        # Leave a clearer message for the user
        print(f"\nERROR: Could not load file {file_path}. Does the file exist and contain an 'Events' TTree?")
        print(f"Error details: {e}")
        return pd.DataFrame()

This function acts as a **Data Aggregator and Orchestrator**. Its job is to manage the loading of **multiple data files** and combine them into a single, unified DataFrame for subsequent analysis.

### 1. Initialization and Orchestration

* **Initialization:** It starts with an empty list, `all_dfs`, which will temporarily hold the processed DataFrame from each individual file.
* **The Input Map:** It receives `data_files_map`, which is typically a Python dictionary where the **keys** describe the type of data (e.g., "DoubleMuon\_RunB") and the **values** are the physical paths to the ROOT files.
* **Looping through Files:** It iterates through this map, treating each file independently.
### 2. File Handling and Delegation

For every existing file, it calls the crucial **`load_data_from_file`** function above. If `load_data_from_file` returns a DataFrame that is **not empty** (`if not df.empty:`), the function appends this successfully loaded and flattened DataFrame to the `all_dfs` list.
### 3. Final Aggregation (Concatenation)

Once the loop is finished, the function brings all the pieces together, using **`pd.concat(all_dfs, ignore_index=True)`**.
    
This takes the list of DataFrames (each containing only one type of lepton, like Muons from Run B, Muons from Run C, etc.) and stacks them vertically to create a single, massive **`final_df`**.

`ignore_index=True` ensures the row indexing is reset from 0 to N across the entire combined data set, preventing duplicate index values from the individual files.
```

In [ ]:

def load_and_flatten_data(data_files_map, max_events):
    """
    Loads data from all specified files and concatenates them into one DataFrame.
    """
    all_dfs = []

    print(f"Attempting to load a maximum of {max_events} events per file...")
    for key, file_name in data_files_map.items():
        if os.path.exists(file_name):
            df = load_data_from_file(file_name, key, max_events)
            if not df.empty:
                all_dfs.append(df)
        else:
            print(f"WARNING: File not found '{file_name}'. Skipped. Check the path.")

    if not all_dfs:
        print("Loading failed: No data file was successfully loaded.")
        return pd.DataFrame()

    # Concatenate all loaded DataFrames
    final_df = pd.concat(all_dfs, ignore_index=True)
    print(f"\nTotal loading successful. Total of {len(final_df)} leptons ready for analysis.")
    return final_df

The next function performs the essential **initial filtering** of the raw lepton data. Its purpose is to discard background particles and poorly measured objects to get only the high-quality leptons suitable for reconstructing signal processes.

### Transverse Momentum ($p_T$)

Most low-$p_T$ leptons are from instrumental noise, so we apply a threshold of $\mathbf{p_T > 5.0}$ **GeV** by creating a new DataFrame (`df_pt_filtered`) containing only rows where the value in the `'pt'` column is greater than $5.0$.

### Pseudorapidity ($\eta$)

Detectors like CMS and ATLAS are designed with specific angular coverages. Most high-quality measurements occur in the **central region** (low $|\eta|$). So we apply a threshold of $\mathbf{|\eta| < 2.5}$ by filtering the previous result (`df_pt_filtered`).

### Isolation Cut

The signal of interest is **prompt leptons** (leptons produced directly from the hard collision or a short-lived particle like the Higgs). A key background source is leptons produced within **hadronic jets** (non-prompt).

**Isolation** (`iso`) is a quantity (typically a ratio) that measures the amount of energy deposited around the lepton track. Prompt leptons are **isolated** (low energy nearby), while leptons inside a jet are **non-isolated** (high energy nearby). It applies a threshold of $\mathbf{iso < 0.3}$.

The final DataFrame is returned after calling **`.reset_index(drop=True)`**. This is important because the filtering steps leave gaps in the DataFrame index. Resetting the index ensures a contiguous sequence of row numbers from $0$ to $N-1$.

In [ ]:
def apply_quality_cuts(df):
    """
    Applies the minimal quality and kinematic cuts to individual leptons.
    This is the first essential filtering step in the H -> 4l analysis.
    """
    initial_count = len(df)

    if initial_count == 0:
        print("The DataFrame is empty, no quality cuts applied.")
        return df

    # A. KINEMATIC CUT ON PT (Transverse Momentum)
    pt_cut = 5.0
    df_pt_filtered = df[df['pt'] > pt_cut]
    print(f"-> pT cut > {pt_cut} GeV: {initial_count - len(df_pt_filtered)} leptons rejected.")

    # B. ACCEPTANCE CUT ON ETA (Pseudorapidity)
    eta_max = 2.5
    df_eta_filtered = df_pt_filtered[np.abs(df_pt_filtered['eta']) < eta_max]
    print(f"-> |eta| cut < {eta_max}: {len(df_pt_filtered) - len(df_eta_filtered)} leptons rejected (after pT).")

    # C. ISOLATION CUT (Rejecting Leptons from Jets)
    iso_cut = 0.3
    df_final = df_eta_filtered[df_eta_filtered['iso'] < iso_cut]
    print(f"-> Isolation Cut (iso < {iso_cut}): {len(df_eta_filtered) - len(df_final)} leptons rejected (after pT and eta).")

    final_count = len(df_final)
    print(f"\nTotal leptons after quality cuts: {final_count}")
    return df_final.reset_index(drop=True)

This function is a **Data Validation and Sanitation** step, critical for ensuring the mathematical integrity of the physics data before complex using `vector` library.

### Unphysical Mass Values
In physics data processing, sometimes the reconstruction algorithm (especially when dealing with charged particles) can result in a **negative mass value**. Since mass ($m$) is derived from the energy ($E$) and momentum ($p$) via $E^2 - p^2 = m^2$, numerical precision errors can occasionally lead to an unphysical $m^2 < 0$. This is mathematically impossible for real particles.

We creates a `negative_mass_mask`, to identify all rows where `df['mass'] < 0`.

Instead of discarding the entire lepton, we set the negative masses to a small positive value, **0.1** (instead of $0.0$ to ensure numerical stability in later calculations). This is a pragmatic approach: we correct the mathematical error while preserving the valid $p_T$, $\eta$, and $\phi$ measurements.

### General Validity and Finite Check

The second part of the function focuses on the strict requirement that all four kinematic variables are numerically well-defined.

* **NaN/Inf Check (Not a Number / Infinity):**
    * It checks the four main kinematic columns (`pt`, `eta`, `phi`, `mass`) simultaneously.
    * `np.isfinite` determines if a number is neither $\text{NaN}$ nor $\pm\infty$.
    * `np.all(..., axis=1)` ensures that a row is kept **only if ALL four** values in that row are finite. This creates the `is_finite` mask. This values will cause issues in `vector` library.
* **$p_T > 0$ Check:**
    * It ensures that the transverse momentum is strictly positive (`pt_ok = df['pt'] > 0`). A lepton with $p_T \le 0$ is physically irrelevant or indicative of a severe reconstruction failure.

### 3. Final Filtering and Cleanup

We combine the two validity checks (`is_finite` and `pt_ok`) into a single `valid_mask` and the DataFrame is filtered to keep only the valid rows (`df_cleaned = df[valid_mask]`).

Finally, as with any filtering operation, it calls `.reset_index(drop=True)` to ensure the cleaned output DataFrame has a clean, continuous row index.

In [ ]:
def clean_kinematic_data(df):
    """
    Corrects negative masses (reconstruction fault) and cleans invalid values
    to ensure that 'vector.array' does not crash.
    """
    initial_count = len(df)
    kinematic_cols = ['pt', 'eta', 'phi', 'mass']

    # 1. CORRECTION: Identify and correct negative masses (the main problem)
    negative_mass_mask = df['mass'] < 0
    num_neg_mass = negative_mass_mask.sum()

    if num_neg_mass > 0:
        # Warning and correction: force unphysical masses to 0.0
        print(f"\n--- MASS CORRECTION: {num_neg_mass} negative masses forced to 0.0 ---")
        df.loc[negative_mass_mask, 'mass'] = 0.1

    # 2. CLEANUP: Mask for NaN/Inf (all columns must be finite)
    is_finite = np.all(np.isfinite(df[kinematic_cols].values), axis=1)

    # 3. CLEANUP: Mask for pT > 0 (mass is now >= 0 thanks to step 1)
    pt_ok = df['pt'] > 0

    # Combined mask for valid data that remains inside
    valid_mask = is_finite & pt_ok

    df_cleaned = df[valid_mask].reset_index(drop=True)
    removed_count = initial_count - len(df_cleaned)

    if removed_count > 0:
        print(f"-> Final cleanup after correction: {removed_count} rows rejected (NaN, Inf, or pT <= 0).")
    else:
        print("-> Final cleanup: No invalid data found after mass correction.")

    return df_cleaned


This function is a **Visual Diagnostics Tool** crucial for validating the filtering steps (like the `apply_quality_cuts` function)

It takes two DataFrames: `df_before` (the raw, unfiltered lepton data) and `df_after` (the high-quality lepton data resulting from the cuts).


This function serves as a **key quality assurance step**, providing clear visual evidence of the data selection process.

In [ ]:
def plot_kinematic_diagnostics(df_before, df_after):
    """
    Plots pT and eta distributions to visualize the effect of the applied cuts.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # --- Plot pT Distribution ---
    axes[0].hist(df_before['pt'], bins=50, range=(0, 100), histtype='step',
                 label='Before cuts', color='blue', density=True)
    axes[0].hist(df_after['pt'], bins=50, range=(0, 100), histtype='stepfilled', alpha=0.6,
                 label=f'After cuts ({len(df_after)} leptons)', color='red', density=True)

    # Vertical line to show the pT > 5 GeV cut
    axes[0].axvline(5.0, color='red', linestyle='--', linewidth=1, label='$p_T > 5 \\text{ GeV}$')

    axes[0].set_yscale('log')
    axes[0].set_xlabel('$p_T$ of Lepton (GeV)')
    axes[0].set_ylabel('Events (Normalized)')
    axes[0].set_title('$p_T$ Distribution (Transverse Momentum)')
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.5)

    # --- Plot eta Distribution ---
    axes[1].hist(df_before['eta'], bins=40, range=(-3.0, 3.0), histtype='step',
                 label='Before cuts', color='blue', density=True)
    axes[1].hist(df_after['eta'], bins=40, range=(-3.0, 3.0), histtype='stepfilled', alpha=0.6,
                 label=f'After cuts ({len(df_after)} leptons)', color='red', density=True)

    # Vertical lines to show the |eta| < 2.5 cut
    axes[1].axvline(2.5, color='red', linestyle='--', linewidth=1, label='$|\\eta| < 2.5$')
    axes[1].axvline(-2.5, color='red', linestyle='--', linewidth=1)

    axes[1].set_xlabel('$\\eta$ of Lepton (Pseudorapidity)')
    axes[1].set_ylabel('Events (Normalized)')
    axes[1].set_title('$\\eta$ Distribution (Acceptance)')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:

# --- 5. MAIN EXECUTION ---
print("--- START OF H -> 4l ANALYSIS (Real Data) ---")

# 1. Load the original data
all_leptons_df_initial = load_and_flatten_data(DATA_FILES, MAX_EVENTS)

if not all_leptons_df_initial.empty:
    # 2. Apply individual quality cuts
    filtered_leptons_df = apply_quality_cuts(all_leptons_df_initial)

    cleaned_leptons_df = clean_kinematic_data(filtered_leptons_df)
    # 4. CREATION OF LORENTZ VECTORS ON ALL FILTERED DATA (The right way)
    print("\nStep 4: Creation of Lorentz Vector (lv) objects on the complete filtered DataFrame...")
    try:
        vector_array_temp = vector.array({
            "pt": cleaned_leptons_df['pt'],
            "eta": cleaned_leptons_df["eta"],
            "phi": cleaned_leptons_df["phi"],
            "mass": cleaned_leptons_df["mass"]
        })

        cleaned_leptons_df['lv'] = vector_array_temp


        print("Success: 'lv' column of Lorentz vectors added to the DataFrame.")
        print(cleaned_leptons_df['lv'].values.dtype)

    except Exception as e:
        print(f"\nCRITICAL ERROR during creation of 'lv' column: {e}")
        print("The analysis cannot continue. Please check the installation or version of 'vector'.")


    # 3. VERIFICATION: Display diagnostics
    plot_kinematic_diagnostics(all_leptons_df_initial, cleaned_leptons_df)

    # 4. Confirm the next step
    print("\n--- STATUS: Quality cuts applied and verified. ---")

    if not cleaned_leptons_df.empty:
        print("\nPreview of filtered leptons (ready for combination):")
        print(cleaned_leptons_df[['event_id', 'pt', 'charge', 'flavor', 'iso']].head())